In [1]:
import os
os.chdir('../..')

In [2]:
from platform import python_version
print(python_version())

3.7.9


In [3]:
import numpy as np
import pandas as pd

import tensorflow as tf

import matplotlib.pyplot as plt

from modules.datgan import DATWGAN

import networkx as nx
import json
import beepy

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [4]:
dataset = 'Chicago'

In [5]:
df = pd.read_csv('../data/{}/data.csv'.format(dataset), index_col=False)

In [6]:
df.head()

,choice,travel_dow,trip_purpose,distance,hh_vehicles,hh_size,hh_bikes,hh_descr,hh_income,gender,age,license,education_level,work_status,departure_time
0,drive,7,HOME_OTHER,3.93477,2,3,3,detached,6,0,30,1,4,PTE,20.166667
1,drive,2,SHOPPING,0.31557,3,3,3,detached,7,0,54,1,5,FTE,17.500000
2,drive,2,SHOPPING,0.28349,1,1,0,detached,3,0,80,1,3,PTE,9.333333
3,drive,2,OTHER,0.69417,2,2,0,detached,5,1,42,1,5,FTE,13.783333
4,passenger,1,SHOPPING,4.30666,2,2,1,detached,4,0,32,0,3,Unemployed,11.566667


In [7]:
if dataset is 'Chicago':
    continuous_columns = ["distance", "age", "departure_time"]
elif dataset is 'LPMC':
    continuous_columns = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [8]:
# personalised graph
graph = nx.DiGraph()

if dataset is 'Chicago':
    graph.add_edges_from([
        ("age", "license"),
        ("age", "education_level"),
        ("gender", "work_status"),
        ("education_level", "work_status"),
        ("education_level", "hh_income"),
        ("work_status", "hh_income"),
        ("hh_income", "hh_descr"),
        ("hh_income", "hh_size"),
        ("hh_size", "hh_vehicles"),
        ("hh_size", "hh_bikes"),
        ("work_status", "trip_purpose"),
        ("trip_purpose", "departure_time"),
        ("trip_purpose", "distance"),
        ("travel_dow", "choice"),
        ("distance", "choice"),
        ("departure_time", "choice"),
        ("hh_vehicles", "choice"),
        ("hh_bikes", "choice"),
        ("license", "choice"),
        # Non necessary links
        ("education_level", "hh_size"),
        ("work_status", "hh_descr"),
        ("work_status", "hh_size"),
        ("hh_income", "hh_bikes"),
        ("hh_income", "hh_vehicles"),
        ("trip_purpose", "choice")
    ])
elif dataset is 'LPMC':
    graph.add_edges_from([
        ("travel_year", "survey_year"),
        ("travel_date", "day_of_week"),
        ("day_of_week", "purpose"),
        ("purpose", "start_time_linear"),
        ("purpose", "cost_driving_con_charge"),
        ("purpose", "distance"),
        ("day_of_week", "driving_traffic_percent"),
        ("day_of_week", "cost_driving_con_charge"),
        ("start_time_linear", "driving_traffic_percent"),
        ("start_time_linear", "cost_driving_con_charge"),
        ("driving_traffic_percent", "cost_driving_con_charge"),
        ("female", "driving_license"),
        ("age", "bus_scale"),
        ("age", "car_ownership"),
        ("age", "driving_license"),
        ("age", "faretype"),
        ("driving_license", "car_ownership"),
        ("car_ownership", "fueltype"),
        ("fueltype", "cost_driving_con_charge"),
        ("fueltype", "cost_driving_fuel"),
        ("distance", "cost_driving_fuel"),
        ("distance", "dur_driving"),
        ("distance", "dur_walking"),
        ("distance", "dur_cycling"),
        ("distance", "dur_pt_access"),
        ("distance", "dur_pt_rail"),
        ("distance", "dur_pt_bus"),
        ("distance", "dur_pt_int"),
        ("dur_pt_bus", "cost_transit"),
        ("dur_pt_rail", "cost_transit"),
        ("pt_n_interchanges", "dur_pt_int"),
        ("pt_n_interchanges", "cost_transit"),
        ("faretype", "cost_transit"),
        ("bus_scale", "cost_transit"),
        ("car_ownership", "travel_mode"),
        ("age", "travel_mode"),
        ("cost_driving_con_charge", "travel_mode"),
        ("driving_traffic_percent", "travel_mode"),
        ("female", "travel_mode"),
        ("purpose", "travel_mode"),
        ("cost_transit", "travel_mode"),
        ("cost_driving_fuel", "travel_mode"),
        ("dur_driving", "travel_mode"),
        ("dur_walking", "travel_mode"),
        ("dur_cycling", "travel_mode"),
        ("dur_pt_access", "travel_mode"),
        ("dur_pt_rail", "travel_mode"),
        ("dur_pt_bus", "travel_mode"),
        ("dur_pt_int", "travel_mode")
    ])
    graph.add_node("travel_month")

In [9]:
output_folder = '../output/' + dataset + '/WGAN_WO/'

In [10]:
datgan = DATWGAN(continuous_columns, max_epoch=1000, batch_size=500, output=output_folder, gpu=0, learning_rate=2e-4)

In [11]:
test = datgan.fit(df, graph)

[1111 12:43:02 @DATSGAN.py:153] Found preprocessed data
[1111 12:43:02 @DATSGAN.py:161] Preprocessed data have been loaded!



[1111 12:43:02 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
[1111 12:43:02 @DATSGANModel.py:205] Creating cell for age (in-edges: 0; ancestors: 0)
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[1111 12:43:02 @registry.py:126] gen/LSTM/age/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[1111 12:43:02 @registry.py:134] gen/LSTM/age/FC output: [500, 50]
[1111 12:43:02 @registry.py:126] gen/LSTM/age/FC_val input: [500, 50]
[1111 12:43:02 @registry.py:134] gen/LSTM/age/FC_val output: [500, 5]
[1111

[1111 12:43:02 @registry.py:126] gen/LSTM/work_status/FC input: [500, 100]
[1111 12:43:02 @registry.py:134] gen/LSTM/work_status/FC output: [500, 50]
[1111 12:43:02 @registry.py:126] gen/LSTM/work_status/FC_prob input: [500, 50]
[1111 12:43:02 @registry.py:134] gen/LSTM/work_status/FC_prob output: [500, 8]
[1111 12:43:02 @registry.py:126] gen/LSTM/work_status/FC_input input: [500, 8]
[1111 12:43:02 @registry.py:134] gen/LSTM/work_status/FC_input output: [500, 100]
[1111 12:43:02 @DATSGANModel.py:205] Creating cell for hh_income (in-edges: 2; ancestors: 4)
[1111 12:43:02 @registry.py:126] gen/LSTM/concat-hh_income/FC_inputs input: [500, 200]
[1111 12:43:02 @registry.py:134] gen/LSTM/concat-hh_income/FC_inputs output: [500, 100]
[1111 12:43:02 @registry.py:126] gen/LSTM/concat-hh_income/FC_states input: [500, 200]
[1111 12:43:02 @registry.py:134] gen/LSTM/concat-hh_income/FC_states output: [500, 100]
[1111 12:43:02 @registry.py:126] gen/LSTM/concat-hh_income/FC_h_outputs input: [500, 200

[1111 12:43:03 @registry.py:126] gen/LSTM/hh_bikes/FC input: [500, 100]
[1111 12:43:03 @registry.py:134] gen/LSTM/hh_bikes/FC output: [500, 50]
[1111 12:43:03 @registry.py:126] gen/LSTM/hh_bikes/FC_prob input: [500, 50]
[1111 12:43:03 @registry.py:134] gen/LSTM/hh_bikes/FC_prob output: [500, 8]
[1111 12:43:03 @registry.py:126] gen/LSTM/hh_bikes/FC_input input: [500, 8]
[1111 12:43:03 @registry.py:134] gen/LSTM/hh_bikes/FC_input output: [500, 100]
[1111 12:43:03 @DATSGANModel.py:205] Creating cell for choice (in-edges: 7; ancestors: 13)
[1111 12:43:03 @registry.py:126] gen/LSTM/concat-choice/FC_inputs input: [500, 700]
[1111 12:43:03 @registry.py:134] gen/LSTM/concat-choice/FC_inputs output: [500, 100]
[1111 12:43:03 @registry.py:126] gen/LSTM/concat-choice/FC_states input: [500, 700]
[1111 12:43:03 @registry.py:134] gen/LSTM/concat-choice/FC_states output: [500, 100]
[1111 12:43:03 @registry.py:126] gen/LSTM/concat-choice/FC_h_outputs input: [500, 700]
[1111 12:43:03 @registry.py:134] 

[1111 12:43:06 @base.py:209] Setup callbacks graph ...







[1111 12:43:06 @utils.py:15] Clip discrim/DISCR_FC_0/FC/W

[1111 12:43:06 @utils.py:15] Clip discrim/DISCR_FC_0/FC/b
[1111 12:43:06 @utils.py:15] Clip discrim/DISCR_FC_0/FC_DIVERSITY/W
[1111 12:43:06 @utils.py:15] Clip discrim/DISCR_FC_0/FC_DIVERSITY/b
[1111 12:43:06 @utils.py:15] Clip discrim/DISCR_FC_0/BN/beta
[1111 12:43:06 @utils.py:15] Clip discrim/DISCR_FC_TOP/W
[1111 12:43:06 @utils.py:15] Clip discrim/DISCR_FC_TOP/b
[1111 12:43:06 @summary.py:46] [MovingAverageSummary] 3 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[1111 12:43:06 @summary.py:93] Summarizing collection 'summaries' of size 4.

[1111 12:43:06 @graph.py:98] Applying collection UPDATE_OPS of 4 ops.

[1111 12:43:07 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[1111 12:43:09 @base.py:236] Initializing the session ...
[1111 12:43:09 

 18%|###############1                                                                      |3/17[00:02<00:09, 1.43it/s]

...


100%|#####################################################################################|17/17[00:00<00:00,37.79it/s]

[1111 12:53:04 @base.py:285] Epoch 999 (global_step 25474) finished, time:0.45 second.
[1111 12:53:04 @saver.py:79] Model saved to ../output/Chicago/WGAN_WO/model\model-25474.
[1111 12:53:04 @monitor.py:467] GAN_loss/d_loss: -0.046316
[1111 12:53:04 @monitor.py:467] GAN_loss/g_loss: -0.10207
[1111 12:53:04 @monitor.py:467] GAN_loss/kl_div: 0.11826
[1111 12:53:04 @monitor.py:467] QueueInput/queue_size: 50
[1111 12:53:04 @base.py:275] Start Epoch 1000 ...



100%|#####################################################################################|17/17[00:00<00:00,39.03it/s]

[1111 12:53:04 @base.py:285] Epoch 1000 (global_step 25499) finished, time:0.436 second.
[1111 12:53:04 @saver.py:79] Model saved to ../output/Chicago/WGAN_WO/model\model-25499.
[1111 12:53:04 @monitor.py:467] GAN_loss/d_loss: -0.047056
[1111 12:53:04 @monitor.py:467] GAN_loss/g_loss: -0.10059
[1111 12:53:04 @monitor.py:467] GAN_loss/kl_div: 0.11139
[1111 12:53:04 @monitor.py:467] QueueInput/queue_size: 50
[1111 12:53:04 @base.py:289] Training has finished!



[1111 12:53:04 @DATSGANModel.py:205] Creating cell for age (in-edges: 0; ancestors: 0)
[1111 12:53:04 @DATSGANModel.py:205] Creating cell for gender (in-edges: 0; ancestors: 0)
[1111 12:53:05 @DATSGANModel.py:205] Creating cell for travel_dow (in-edges: 0; ancestors: 0)
[1111 12:53:05 @DATSGANModel.py:205] Creating cell for license (in-edges: 1; ancestors: 1)
[1111 12:53:05 @DATSGANModel.py:205] Creating cell for education_level (in-edges: 1; ancestors: 1)
[1111 12:53:05 @DATSGANModel.py:205] Creating cell for work_status (in-edges: 2; ancestors: 3)
[1111 12:53:05 @DATSGANModel.py:205] Creating cell for hh_income (in-edges: 2; ancestors: 4)
[1111 12:53:05 @DATSGANModel.py:205] Creating cell for trip_purpose (in-edges: 1; ancestors: 4)
[1111 12:53:05 @DATSGANModel.py:205] Creating cell for hh_descr (in-edges: 2; ancestors: 5)
[1111 12:53:05 @DATSGANModel.py:205] Creating cell for hh_size (in-edges: 3; ancestors: 5)
[1111 12:53:05 @DATSGANModel.py:205] Creating cell for departure_time (

In [12]:
datgan.save('trained', force=True)

[1111 12:53:22 @DATSGAN.py:362] Model saved successfully.


In [13]:
beepy.beep(6)